In [2]:
import eikon as ek
ek.set_app_key('9a430e13c8a944e9894743c7bd608b7e41a432be')

In [22]:
import eikon as ek  # the Eikon Python wrapper package
import numpy as np  # NumPy
import pandas as pd  # pandas
import cufflinks as cf  # Cufflinks
from sklearn.svm import SVC  # sckikit-learn
from sklearn.model_selection import train_test_split
import configparser as cp
from collections import Counter
from datetime import datetime
import statistics
import datetime as dt

# S&P1500 Firms

In [4]:
df,er = ek.get_data(['0#.SPX','0#.SPMIDSM'],['TR.NumberOfAnalysts','TR.RecMean','TR.CompanyMarketCap'],
                    {'frq': 'FY', 'SDate':'2020-01-01', 'EDate':'2020-12-31'})
df.head(10)


,Instrument,Number of Analysts,Recommendation - Mean (1-5),Company Market Cap
0,POOL.OQ,9,2.55556,1.495907e+10
1,CHRW.OQ,23,2.81818,1.276126e+10
2,AJG.N,13,2.23077,2.379215e+10
3,CNP.N,18,2.37500,1.178993e+10
4,AMCR.N,8,2.62500,1.846103e+10
5,WM.N,16,2.33333,4.983795e+10
6,PRGO.N,12,2.72727,6.103891e+09
7,BA.N,24,2.82609,1.246514e+11
8,FOX.OQ,27,2.53846,1.736921e+10
9,LIN.N,23,2.38095,1.379054e+11


# Identifying the number of listed firms

In [5]:
index=df.index
no_of_listed_firms=len(index)
print(no_of_listed_firms)

1509


# Identifying the number of covered firms

In [6]:
df.isnull().sum()      #to check for any na values

Instrument                      0
Number of Analysts              1
Recommendation - Mean (1-5)    32
Company Market Cap              4
dtype: int64

In [7]:
df['Number of Analysts'] = df['Number of Analysts'].fillna(0) #to replace na values with 0
no_of_covered_firms = 0 
for i in range(len(df)):
    if df['Number of Analysts'][i] != 0:
        no_of_covered_firms += 1

no_of_covered_firms

1474

# Identifying covered firms as a % of listed firms

In [8]:
col4 = no_of_covered_firms/no_of_listed_firms*100
col4

97.6805831676607

# Identifying covered firms' market cap as a % of total listed firms' market cap

In [9]:
sumoflisted =df['Company Market Cap'].sum(skipna=bool)
sumofcovered =df.loc[df['Number of Analysts'] != 0,'Company Market Cap'].sum(skipna=bool)
col5=sumofcovered/sumoflisted*100
col5

99.61393528180999

# Identifying Analysts Per Covered firm Mean and Median

In [10]:
df['Number of Analysts'] = df['Number of Analysts'].fillna(0) #to replace na values with 0

for i in range(len(df)):
    if df['Number of Analysts'][i] != 0:
        mean6=df['Number of Analysts'].mean()
        median7=df['Number of Analysts'].median()
    
print(mean6)
print(median7)

11.905235255135851
10.0


# Identifying Covered Firms per Analyst Mean and Median

In [11]:
df1,er1 = ek.get_data(['0#.SPX','0#.SPMIDSM'],['TR.AnalystCode'],
                    {'frq': 'FY', 'SDate':'2020-01-01', 'EDate':'2020-12-31'})    
df1.head(20)                           #Unique analyst ID to check how many firms does each analyst cover for

,Instrument,Analyst - Analyst Code
0,POOL.OQ,1RER
1,POOL.OQ,7W0X
2,POOL.OQ,5Y1B
3,POOL.OQ,35VW
4,POOL.OQ,2SOA
5,POOL.OQ,7NMO
6,POOL.OQ,3UE1
7,POOL.OQ,4EH7
8,POOL.OQ,4WV2
9,CHRW.OQ,6BCW


In [12]:
analysts=df1['Instrument'].groupby(df1['Analyst - Analyst Code']).count()

analysts.head()          #firms are grouped by analyst code: how many firms does one code cover?

Analyst - Analyst Code
        21
166M     2
17R0    12
19WU    17
1BEM     9
Name: Instrument, dtype: int64

In [13]:
#What does 3 signify? Is that for unknown analyst codes?

In [14]:
mean8=analysts.mean()
median9=analysts.median()
print(mean8)
print(median9)               

8.661372950819672
7.0


# OUTPUT:

In [15]:
data = {'No. of Listed Firms': [no_of_listed_firms] , 'No. of Covered Firms' : [no_of_covered_firms],
        'As a % of Listed Firms': [col4],'Market Cap As % of Market':[col5],'Analysts per Covered Firm(Mean)':[mean6],
        'Analysts per Covered Firm(Median)':[median7],'Covered Firm per Analyst(Mean)':[mean8],
        'Covered Firm per Analyst(Median)':[median9],'Number of Analysts':[df['Number of Analysts'].sum(skipna=bool)],
        'Average Rating':[df['Recommendation - Mean (1-5)'].mean(skipna=bool)]}

In [16]:
table = pd.DataFrame(data)
years = [2020]
table.index = years
table

,No. of Listed Firms,No. of Covered Firms,As a % of Listed Firms,Market Cap As % of Market,Analysts per Covered Firm(Mean),Analysts per Covered Firm(Median),Covered Firm per Analyst(Mean),Covered Firm per Analyst(Median),Number of Analysts,Average Rating
2020,1509,1474,97.680583,99.613935,11.905235,10.0,8.661373,7.0,17965,2.403828


# S&P1500 Final Output

In [17]:
years = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]

data = {'No. of Listed Firms': [] , 'No. of Covered Firms' : [],
    'As a % of Listed Firms': [],'Market Cap As % of Market':[],'Analysts per Covered Firm(Mean)':[],
    'Analysts per Covered Firm(Median)':[],'Covered Firm per Analyst(Mean)':[],
    'Covered Firm per Analyst(Median)':[],'Number of Analysts':[],
    'Average Rating':[]}

table = pd.DataFrame(data)

for year in years:
    df,er = ek.get_data(['0#.SPX','0#.SPMIDSM'],['TR.NumberOfAnalysts','TR.RecMean','TR.CompanyMarketCap'],
                    {'frq': 'FY', 'SDate':'{}-01-01'.format(year), 'EDate':'{}-12-31'.format(year)})
    
    index=df.index
    no_of_listed_firms=len(index)
    
    df['Number of Analysts'] = df['Number of Analysts'].fillna(0) #to replace na values with 0
    no_of_covered_firms = 0 
    for i in range(len(df)):
        if df['Number of Analysts'][i] != 0:
            no_of_covered_firms += 1
            
    col4 = no_of_covered_firms/no_of_listed_firms*100
    
    sumoflisted =df['Company Market Cap'].sum(skipna=bool)
    sumofcovered =df.loc[df['Number of Analysts'] != 0,'Company Market Cap'].sum(skipna=bool)
    col5=sumofcovered/sumoflisted*100
    
    df['Number of Analysts'] = df['Number of Analysts'].fillna(0) #to replace na values with 0

    for i in range(len(df)):
        if df['Number of Analysts'][i] != 0:
            mean6=df['Number of Analysts'].mean()
            median7=df['Number of Analysts'].median()      
    
    df1,er1 = ek.get_data(['0#.SPX','0#.SPMIDSM'],['TR.AnalystCode'],
                    {'frq': 'FY', 'SDate':'{}-01-01'.format(year), 'EDate':'{}-12-31'.format(year)}) 
    
    analysts=df1['Instrument'].groupby(df1['Analyst - Analyst Code']).count()
    mean8=analysts.mean()
    median9=analysts.median()
    
    ##Output##
    data = {'No. of Listed Firms': [no_of_listed_firms] , 'No. of Covered Firms' : [no_of_covered_firms],
        'As a % of Listed Firms': [col4],'Market Cap As % of Market':[col5],'Analysts per Covered Firm(Mean)':[mean6],
        'Analysts per Covered Firm(Median)':[median7],'Covered Firm per Analyst(Mean)':[mean8],
        'Covered Firm per Analyst(Median)':[median9],'Number of Analysts':[df['Number of Analysts'].sum(skipna=bool)],
        'Average Rating':[df['Recommendation - Mean (1-5)'].mean(skipna=bool)]}
    table2 = pd.DataFrame(data, index=[year])
    table = pd.concat([table, table2])

table

,No. of Listed Firms,No. of Covered Firms,As a % of Listed Firms,Market Cap As % of Market,Analysts per Covered Firm(Mean),Analysts per Covered Firm(Median),Covered Firm per Analyst(Mean),Covered Firm per Analyst(Median),Number of Analysts,Average Rating
2010,1511.0,1151.0,76.174719,94.098716,8.859034,7.0,8.661373,7.0,13386.0,2.311071
2011,1513.0,1212.0,80.105750,96.348283,10.034369,8.0,8.661373,7.0,15182.0,2.299245
2012,1509.0,1240.0,82.173625,96.175145,10.469185,8.0,8.661373,7.0,15798.0,2.290217
2013,1510.0,1273.0,84.304636,96.222742,10.889404,9.0,8.661373,7.0,16443.0,2.323754
2014,1509.0,1309.0,86.746190,96.855180,11.480451,9.0,8.661373,7.0,17324.0,2.329091
2015,1509.0,1367.0,90.589795,95.862840,11.632207,9.0,8.661373,7.0,17553.0,2.304355
2016,1512.0,1406.0,92.989418,99.170679,11.921296,9.0,8.661373,7.0,18025.0,2.303550
2017,1510.0,1422.0,94.172185,98.822063,12.036424,9.0,8.661373,7.0,18175.0,2.393667
2018,1508.0,1442.0,95.623342,99.619730,11.861406,9.0,8.661373,7.0,17887.0,2.357617
2019,1509.0,1465.0,97.084162,99.643256,12.031146,10.0,8.661373,7.0,18155.0,2.351676
